In [1]:
import lsdb

In [2]:
from dask.distributed import Client
client = Client(n_workers=70, memory_limit="25GB")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')
from tqdm import tqdm

In [4]:
%%time
fgk_object = lsdb.read_hipscat("/nvme/users/atzanida/tmp/sample_final_starhorse_hips")
fgk_table = fgk_object.compute()

CPU times: user 23.7 s, sys: 53.3 s, total: 1min 16s
Wall time: 1min 10s


In [5]:
# Select only RA and DEC columns
new_fgk = fgk_table[['RA_ICRS_StarHorse', 'DE_ICRS_StarHorse', 'ps1_objid_ztf_dr14']]

In [6]:
# Select only small box between 70 - 90 de
small_box = new_fgk.query("DE_ICRS_StarHorse < 100 and DE_ICRS_StarHorse > 80")

In [7]:
small_box = small_box.reset_index()

In [8]:
small_box = small_box[['RA_ICRS_StarHorse', 'DE_ICRS_StarHorse', 'ps1_objid_ztf_dr14']]

In [9]:
small_box.head(1)

,RA_ICRS_StarHorse,DE_ICRS_StarHorse,ps1_objid_ztf_dr14
0,88.691641,80.000625,204000886915911621


In [10]:
len(small_box)

292168

In [11]:
!rm /nvme/users/atzanida/tmp/StarHorse_Dec9080.parquet

In [12]:
%%time
# Store small FGK StarHorse
small_box.to_parquet("/nvme/users/atzanida/tmp/StarHorse_Dec9080.parquet", engine='pyarrow')

CPU times: user 196 ms, sys: 155 ms, total: 351 ms
Wall time: 252 ms


In [13]:
%%time
final_tab_slab1 = lsdb.from_dataframe(small_box,
                                ra_column="RA_ICRS_StarHorse", 
                                dec_column="DE_ICRS_StarHorse", 
                                threshold=8_000)

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 13.69 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 13.69 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 8min 34s, sys: 3min 3s, total: 11min 37s
Wall time: 8min 7s


In [14]:
final_tab_slab1

,RA_ICRS_StarHorse,DE_ICRS_StarHorse,ps1_objid_ztf_dr14,Norder,Dir,Npix
npartitions=77,,,,,,
1103381908705771520,float64,float64,int64,uint8,uint64,uint64
1107885508333142016,...,...,...,...,...,...
...,...,...,...,...,...,...
4609434218613702656,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...


In [15]:
# Load ZTF DR14 sources
ztf_sources = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_zource")

### Custom functions

In [24]:
# magic for real time editing of .py files
%load_ext autoreload
%autoreload 2

# Custom functions and tools 
import sys
sys.path.insert(1, '../../../dipper')

import tools as my_tools
import dipper as dip_pipeline
import models as dip_models
from evaluate import evaluate as evaluate
from evaluate import evaluate_updated
from gpmcmc import model_gp
from evaluate import half_eval as half_eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
evaluate_updated

<function evaluate.evaluate_updated(time_cat, mag_cat, mag_err_cat, flag_cat, band_cat, custom_cols=['Nphot', 'biweight_scale', 'frac_above_2_sigma', 'Ndips', 'rate', 'chi2dof', 'skew', 'kurtosis', 'mad', 'stetson_i', 'stetson_j', 'stetson_k', 'invNeumann', 'best_dip_power', 'best_dip_time_loc', 'best_dip_start', 'best_dip_end', 'best_dip_dt', 'best_dip_ndet', 'lc_score'], min_phot=10)>

# initalize tape

In [28]:
import dask.dataframe as dd
from tape import Ensemble, ColumnMapper
# Initialize an Ensemble
ens = Ensemble(client=client)

In [29]:
ens.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 69.85 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42321,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: 14 minutes ago,Total memory: 69.85 GiB
Comm: tcp://127.0.0.1:39150,Total threads: 2
Dashboard: http://127.0.0.1:40289/status,Memory: 23.28 GiB
Nanny: tcp://127.0.0.1:39115,


In [19]:
%%time
_sources = final_tab_slab1.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid")

CPU times: user 348 ms, sys: 20.7 ms, total: 369 ms
Wall time: 358 ms


In [32]:
# ColumnMapper Establishes which table columns map to timeseries quantities
colmap = ColumnMapper(
        id_col='_hipscat_index',
        time_col='mjd',
        flux_col='mag',
        err_col='magerr',
        band_col='band',
      )

ens.from_dask_dataframe(
    source_frame=_sources._ddf,
    object_frame=final_tab_slab1._ddf,
    column_mapper=colmap,
    sync_tables=False, # Avoid doing an initial sync
    sorted=True, # If the input data is already sorted by the chosen index
    sort=False,
)

In [33]:
# feature evaluation 
column_names = ['Nphot',
    'biweight_scale',
    'frac_above_2_sigma', # in deviation
    'Ndips',
    'rate',
    'chi2dof',
    'skew', 
    'kurtosis',
    'mad',
    'stetson_i',
    'stetson_j',
    'stetson_k',
    'invNeumann',    
    'best_dip_power',
    'best_dip_time_loc',
    'best_dip_start',
    'best_dip_end',
    'best_dip_dt',
    'best_dip_ndet',
    'lc_score']

# Define DataFrame with loc and scale as meta
my_meta = pd.DataFrame(columns=column_names, dtype=float)

In [34]:
%%time
# apply calc_biweight function
batch_cal = ens.batch(
    evaluate_updated,
    'mjd_ztf_zource', 'mag_ztf_zource', 
    'magerr_ztf_zource', 'catflags_ztf_zource',
    'band_ztf_zource',
    meta=my_meta,
    use_map=True)

Using generated label, result_1, for a batch result.
CPU times: user 5.97 ms, sys: 0 ns, total: 5.97 ms
Wall time: 5.81 ms


In [35]:
%%time
my_batch = batch_cal.compute()

Task exception was never retrieved
future: <Task finished name='Task-503401' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-503342' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finis

Task exception was never retrieved
future: <Task finished name='Task-503359' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-503360' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finis

Task exception was never retrieved
future: <Task finished name='Task-503377' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-503378' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finis

Task exception was never retrieved
future: <Task finished name='Task-503395' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-503396' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finis

Task exception was never retrieved
future: <Task finished name='Task-503414' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-503415' coro=<Client._gather.<locals>.wait() done, defined at /epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:2209> exception=AllExit()>
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py", line 2218, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finis

...

2024-04-14 18:59:31,353 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:35715 (pid=113150) exceeded 95% memory budget. Restarting...
2024-04-14 18:59:33,474 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:35715
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
    response = await get_data_from_worker(
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/worker.py", line 2863, in get_data_from_worker
    response = a

2024-04-14 19:01:38,224 - distributed.nanny - WARNING - Restarting worker
2024-04-14 19:01:55,833 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-04-14 19:01:58,030 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 16.39 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:01:59,982 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 18.73 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:02:02,850 - distributed.worker.memory - WARNING - Worker is at 73% memory usage. Resuming worker. Process memory: 17.06 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:02:18,665 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:38805 (pid=

2024-04-14 19:19:40,020 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-04-14 19:19:45,184 - distributed.worker.memory - WARNING - Worker is at 86% memory usage. Pausing worker.  Process memory: 20.07 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:19:46,465 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:37470 (pid=114576) exceeded 95% memory budget. Restarting...
2024-04-14 19:19:48,142 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:42645 -> tcp://127.0.0.1:37470
Traceback (most recent call last):
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: 

2024-04-14 19:20:28,675 - distributed.worker.memory - WARNING - Worker is at 86% memory usage. Pausing worker.  Process memory: 20.18 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:20:31,524 - distributed.worker.memory - WARNING - Worker is at 53% memory usage. Resuming worker. Process memory: 12.57 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:20:46,968 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 14.05 GiB -- Worker memory limit: 23.28 GiB
2024-04-14 19:20:59,294 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanage

KeyboardInterrupt: 